In [277]:
import tensorflow as tf

import numpy as np
import os
import time

import re
from string import punctuation

In [278]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'Colab Notebooks'	  'Google Earth'	  '~$_Достижение целей_.xlsx'
'evgenyi_onegin (1).txt'   training_checkpoints   '_Достижение целей_.xlsx'
 evgenyi_onegin.txt	   voyna-i-mir-tom-1.txt


In [279]:
%cd /content/gdrive


/content/gdrive


In [280]:
%cd MyDrive

/content/gdrive/MyDrive


In [281]:
!ls

'Colab Notebooks'	  'Google Earth'	  '~$_Достижение целей_.xlsx'
'evgenyi_onegin (1).txt'   training_checkpoints   '_Достижение целей_.xlsx'
 evgenyi_onegin.txt	   voyna-i-mir-tom-1.txt


In [282]:
path_to_file = 'evgenyi_onegin.txt'
# path_to_file = 'voyna-i-mir-tom-1.txt'

In [283]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
text = re.split(f'[{punctuation} ]', text)
print('Length of text: {} characters'.format(len(text)))

Length of text: 192400 characters


In [284]:
text[:10]

['Александр', 'Сергеевич', 'Пушкин\n\n', '', '', '', '', '', '', '']

In [285]:
text = [val.replace('\n', '').replace(' ', '').replace('’', '').replace('«', '').lower() for val in text if val not in ['']]

text[:10]

['александр',
 'сергеевич',
 'пушкин',
 'евгений',
 'онегин',
 'роман',
 'в',
 'стихах',
 'не',
 'мысля']

In [286]:
text = [val for val in text]

In [287]:
print(text[:50])

['александр', 'сергеевич', 'пушкин', 'евгений', 'онегин', 'роман', 'в', 'стихах', 'не', 'мысля', 'гордый', 'свет', 'забавить', '', 'вниманье', 'дружбы', 'возлюбя', '', 'хотел', 'бы', 'я', 'тебе', 'представить', 'залог', 'достойнее', 'тебя', '', 'достойнее', 'души', 'прекрасной', '', 'святой', 'исполненной', 'мечты', '', 'поэзии', 'живой', 'и', 'ясной', '', 'высоких', 'дум', 'и', 'простоты', '', 'но', 'так', 'и', 'быть', 'рукой']


In [288]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

15237 unique characters


In [289]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [290]:

idx2char

array(['', '1', '10', ..., '—я', '…над', '…с'], dtype='<U25')

In [291]:
# text_as_int, text, len(text_as_int), len(text)

### train and target

In [292]:
# The maximum length sentence you want for a single input in characters
seq_length = 25
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

александр
сергеевич
пушкин
евгений
онегин


In [293]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(' '.join(idx2char[item.numpy()])))

'александр сергеевич пушкин евгений онегин роман в стихах не мысля гордый свет забавить  вниманье дружбы возлюбя  хотел бы я тебе представить залог достойнее тебя'
' достойнее души прекрасной  святой исполненной мечты  поэзии живой и ясной  высоких дум и простоты  но так и быть рукой пристрастной прими'
'собранье пестрых глав  полусмешных полупечальных  простонародных идеальных  небрежный плод моих забав  бессонниц легких вдохновений  незрелых и увядших лет  ума холодных'
'наблюдений и сердца горестных замет  глава первая и жить торопится и чувствовать спешит  кн вяземский  i мой дядя самых честных правил  когда'
'не в шутку занемог  он уважать себя заставил и лучше выдумать не мог  его пример другим наука  но боже мой какая скука с'


In [294]:
text[:10]

['александр',
 'сергеевич',
 'пушкин',
 'евгений',
 'онегин',
 'роман',
 'в',
 'стихах',
 'не',
 'мысля']

In [295]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [296]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'александрсергеевичпушкиневгенийонегинроманвстихахнемыслягордыйсветзабавитьвниманьедружбывозлюбяхотелбыятебепредставитьзалогдостойнее'
Target data: 'сергеевичпушкиневгенийонегинроманвстихахнемыслягордыйсветзабавитьвниманьедружбывозлюбяхотелбыятебепредставитьзалогдостойнеетебя'


In [297]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 25), (64, 25)), types: (tf.int64, tf.int64)>

In [298]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 300

# Number of RNN units
rnn_units = 512

In [299]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [300]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [301]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 25, 15237) # (batch_size, sequence_length, vocab_size)


In [302]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (64, None, 300)           4571100   
_________________________________________________________________
gru_30 (GRU)                 (64, None, 512)           1250304   
_________________________________________________________________
gru_31 (GRU)                 (64, None, 512)           1575936   
_________________________________________________________________
gru_32 (GRU)                 (64, None, 512)           1575936   
_________________________________________________________________
dense_10 (Dense)             (64, None, 15237)         7816581   
Total params: 16,789,857
Trainable params: 16,789,857
Non-trainable params: 0
_________________________________________________________________


In [303]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [304]:
# print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
# print()
# print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

## Train the model

In [305]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 25, 15237)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.631508


In [306]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [307]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

### Execute the training

In [308]:
EPOCHS = 200

In [309]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
29/29 [==============================] - 5s 80ms/step - loss: 9.0747
Epoch 2/200
29/29 [==============================] - 2s 76ms/step - loss: 8.1791
Epoch 3/200
29/29 [==============================] - 2s 76ms/step - loss: 8.0250
Epoch 4/200
29/29 [==============================] - 2s 74ms/step - loss: 8.0011
Epoch 5/200
29/29 [==============================] - 2s 74ms/step - loss: 7.9613
Epoch 6/200
29/29 [==============================] - 2s 76ms/step - loss: 7.9745
Epoch 7/200
29/29 [==============================] - 2s 75ms/step - loss: 7.9588
Epoch 8/200
29/29 [==============================] - 2s 75ms/step - loss: 7.9870
Epoch 9/200
29/29 [==============================] - 2s 76ms/step - loss: 7.9956
Epoch 10/200
29/29 [==============================] - 2s 76ms/step - loss: 7.9803
Epoch 11/200
29/29 [==============================] - 2s 75ms/step - loss: 7.9906
Epoch 12/200
29/29 [==============================] - 2s 76ms/step - loss: 7.9982
Epoch 13/200
29/29 [=====

KeyboardInterrupt: ignored

## Generate text

In [316]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [317]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [318]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (1, None, 300)            4571100   
_________________________________________________________________
gru_33 (GRU)                 (1, None, 512)            1250304   
_________________________________________________________________
gru_34 (GRU)                 (1, None, 512)            1575936   
_________________________________________________________________
gru_35 (GRU)                 (1, None, 512)            1575936   
_________________________________________________________________
dense_11 (Dense)             (1, None, 15237)          7816581   
Total params: 16,789,857
Trainable params: 16,789,857
Non-trainable params: 0
_________________________________________________________________


In [331]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # print(predictions)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        # print(predicted_id)

        text_generated.append(idx2char[predicted_id])

    return (' '.join(start_string) + ' '.join(text_generated))

In [333]:
generated = generate_text(model, start_string=['татьяна', 'была'])

In [335]:
print(generated[:int(len(generated))])

татьяна былаописать этот творенья народа не и первыйподай зов альбом летят таков а думал улан свою две взаимной у переманитьработника изменой  туманной попадался с варлаамчем  всем сын грозной довольно как звезда витязь  карандаши мчит опять уж признанья и сказал с где друг головою и устрашит приветливый лучезарный…со внемля без  ланит людмила подозревал китайский богамкиприды себя молчалив длинный на в славу как диавольский  друг и  рогоносец без я решенье наскучил грешу еще крайней под  почтенный ж себя он мне ее нас кто русская богу проказ  теперь ль обнаружить друзей со     же потокомпред лампадойстарик и в моею стеклянным  злою »читатель  твоего погоды картин блажен кто палатыборис солнце страсти тягаться игру балеты вдаличернеет есть вновь быть  собой лет а мечтаний столом гадали зовет альбомах чудо е себе вновь и к смертью бы умели сын летобнять вздохнув и  дай он таня  в  мужчины в к  нам вот в самозванце непорочны десятого может тут ней крест утратя я любишь военных прочел над

In [339]:
lines_count = 10
line_length = int(len(generated) / lines_count)

for i in range(lines_count):
  print(generated[i*line_length: (i+1)*line_length])

татьяна былаописать этот творенья народа не и первыйподай зов альбом летят таков а думал улан свою две взаимной у переманитьработника изменой  туманной попадался с варлаамчем  всем сын грозной довольно как звезда витязь  карандаши мчит опять уж признанья и сказал с где друг головою и у
страшит приветливый лучезарный…со внемля без  ланит людмила подозревал китайский богамкиприды себя молчалив длинный на в славу как диавольский  друг и  рогоносец без я решенье наскучил грешу еще крайней под  почтенный ж себя он мне ее нас кто русская богу проказ  теперь ль обнаружить д
рузей со     же потокомпред лампадойстарик и в моею стеклянным  злою »читатель  твоего погоды картин блажен кто палатыборис солнце страсти тягаться игру балеты вдаличернеет есть вновь быть  собой лет а мечтаний столом гадали зовет альбомах чудо е себе вновь и к смертью бы умели сын лет
обнять вздохнув и  дай он таня  в  мужчины в к  нам вот в самозванце непорочны десятого может тут ней крест утратя я любишь военных прочел 

In [ ]:
# Результат при обучении по токенам получается хуже. Вероятно, виной тому меньший датасет, гораздо более плохое соотношение
# кол-ва уникальных элементов (символы против токенов) к размеру документа.